In [1]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-
"""
Created on Sun Oct 30 20:11:19 2016

@author: stephen
"""
 
from __future__ import print_function
 
from keras.models import Model
from keras.utils import np_utils
import numpy as np
import pandas as pd
import keras 
from keras.callbacks import ReduceLROnPlateau
      
def readucr(filename):
    data = np.loadtxt(filename, delimiter = ',')
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 2000
foder = u"E:/Jupyter File/UCR_TS_Archive_2015/"

#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'Gun_Point', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist  = ['Adiac']
for each in flist:
    fname = each
    x_train, y_train = readucr(foder+fname+'/'+fname+'_TRAIN')
    x_test, y_test = readucr(foder+fname+'/'+fname+'_TEST')
    nb_classes = len(np.unique(y_test))
    batch_size = min(x_train.shape[0]/10, 16)
    
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    
    
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))

    x = keras.layers.Input(x_train.shape[1:])
#    drop_out = Dropout(0.2)(x)
    conv1 = keras.layers.Conv2D(128, 16, 1, border_mode='same')(x)
    conv1 = keras.layers.normalization.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation('relu')(conv1)
    
#    drop_out = Dropout(0.2)(conv1)
    conv2 = keras.layers.Conv2D(256, 8, 1, border_mode='same')(conv1)
    conv2 = keras.layers.normalization.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)
    
#    drop_out = Dropout(0.2)(conv2)
    conv3 = keras.layers.Conv2D(256, 4, 1, border_mode='same')(conv2)
    conv3 = keras.layers.normalization.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)
    
    #    drop_out = Dropout(0.2)(conv2)
    conv4 = keras.layers.Conv2D(256, 2, 1, border_mode='same')(conv3)
    conv4 = keras.layers.normalization.BatchNormalization()(conv4)
    conv4 = keras.layers.Activation('relu')(conv4)
    
    #    drop_out = Dropout(0.2)(conv2)
    conv5 = keras.layers.Conv2D(128, 1, 1, border_mode='same')(conv4)
    conv5 = keras.layers.normalization.BatchNormalization()(conv5)
    conv5 = keras.layers.Activation('relu')(conv5)    
    
    full = keras.layers.pooling.GlobalAveragePooling2D()(conv5)    
    out = keras.layers.Dense(nb_classes, activation='softmax')(full)
    
    
    model = Model(input=x, output=out)
     
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    hist = model.fit(x_train, Y_train, batch_size=batch_size, nb_epoch=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks = [reduce_lr])
    #Print the testing results which has the lowest training loss.
    log = pd.DataFrame(hist.history)
    print(log.loc[log['loss'].idxmin]['loss'], log.loc[log['loss'].idxmin]['val_acc'])

Using TensorFlow backend.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (16, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (8, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:68: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (4, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:73: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (2, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:78: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (1, 1), padding="same")`
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
C:\Anaconda3\lib\site-packages\ipyk

Train on 390 samples, validate on 391 samples
Epoch 1/2000
390/390 [==============================] - 8s - loss: 3.5734 - acc: 0.0744 - val_loss: 3.6006 - val_acc: 0.0870
Epoch 2/2000
390/390 [==============================] - 7s - loss: 3.2385 - acc: 0.1103 - val_loss: 3.6658 - val_acc: 0.0281
Epoch 3/2000
390/390 [==============================] - 8s - loss: 3.0386 - acc: 0.2026 - val_loss: 4.5782 - val_acc: 0.0281
Epoch 4/2000
390/390 [==============================] - 8s - loss: 2.8999 - acc: 0.2410 - val_loss: 7.0683 - val_acc: 0.0281
Epoch 5/2000
390/390 [==============================] - 7s - loss: 2.7651 - acc: 0.2692 - val_loss: 9.4979 - val_acc: 0.0281
Epoch 6/2000
390/390 [==============================] - 7s - loss: 2.6244 - acc: 0.3051 - val_loss: 10.7721 - val_acc: 0.0281
Epoch 7/2000
390/390 [==============================] - 7s - loss: 2.5003 - acc: 0.3462 - val_loss: 9.8013 - val_acc: 0.0281
Epoch 8/2000
390/390 [==============================] - 7s - loss: 2.4198 - ac

390/390 [==============================] - 7s - loss: 0.7258 - acc: 0.8077 - val_loss: 2.2579 - val_acc: 0.3939
Epoch 66/2000
390/390 [==============================] - 7s - loss: 0.7266 - acc: 0.7795 - val_loss: 1.6515 - val_acc: 0.4885
Epoch 67/2000
390/390 [==============================] - 7s - loss: 0.7883 - acc: 0.7872 - val_loss: 1.9083 - val_acc: 0.5038
Epoch 68/2000
390/390 [==============================] - 7s - loss: 0.8026 - acc: 0.7385 - val_loss: 2.2631 - val_acc: 0.4348
Epoch 69/2000
390/390 [==============================] - 8s - loss: 0.6883 - acc: 0.7949 - val_loss: 2.5397 - val_acc: 0.3683
Epoch 70/2000
390/390 [==============================] - 7s - loss: 0.6660 - acc: 0.7974 - val_loss: 2.7525 - val_acc: 0.3555
Epoch 71/2000
390/390 [==============================] - 7s - loss: 0.6453 - acc: 0.8103 - val_loss: 1.3167 - val_acc: 0.5448
Epoch 72/2000
390/390 [==============================] - 7s - loss: 0.6871 - acc: 0.8051 - val_loss: 2.1476 - val_acc: 0.4476
Epoch 

390/390 [==============================] - 7s - loss: 0.4076 - acc: 0.8821 - val_loss: 1.5133 - val_acc: 0.5217
Epoch 130/2000
390/390 [==============================] - 7s - loss: 0.4154 - acc: 0.8641 - val_loss: 1.8683 - val_acc: 0.4373
Epoch 131/2000
390/390 [==============================] - 7s - loss: 0.4311 - acc: 0.8744 - val_loss: 0.9682 - val_acc: 0.6905
Epoch 132/2000
390/390 [==============================] - 7s - loss: 0.3449 - acc: 0.8974 - val_loss: 1.1802 - val_acc: 0.6113
Epoch 133/2000
390/390 [==============================] - 7s - loss: 0.3567 - acc: 0.8897 - val_loss: 1.3479 - val_acc: 0.6138
Epoch 134/2000
390/390 [==============================] - 7s - loss: 0.3445 - acc: 0.9000 - val_loss: 1.1797 - val_acc: 0.6138
Epoch 135/2000
390/390 [==============================] - 7s - loss: 0.3489 - acc: 0.8974 - val_loss: 1.3518 - val_acc: 0.6036
Epoch 136/2000
390/390 [==============================] - 7s - loss: 0.4272 - acc: 0.8564 - val_loss: 1.4436 - val_acc: 0.5652

390/390 [==============================] - 7s - loss: 0.1979 - acc: 0.9385 - val_loss: 0.8593 - val_acc: 0.7340
Epoch 194/2000
390/390 [==============================] - 7s - loss: 0.1913 - acc: 0.9436 - val_loss: 1.3019 - val_acc: 0.6343
Epoch 195/2000
390/390 [==============================] - 7s - loss: 0.1879 - acc: 0.9487 - val_loss: 0.5990 - val_acc: 0.8056
Epoch 196/2000
390/390 [==============================] - 7s - loss: 0.1914 - acc: 0.9385 - val_loss: 0.8880 - val_acc: 0.7161
Epoch 197/2000
390/390 [==============================] - 7s - loss: 0.2141 - acc: 0.9359 - val_loss: 0.9050 - val_acc: 0.7519
Epoch 198/2000
390/390 [==============================] - 7s - loss: 0.2260 - acc: 0.9103 - val_loss: 0.7076 - val_acc: 0.7724
Epoch 199/2000
390/390 [==============================] - 7s - loss: 0.2262 - acc: 0.9385 - val_loss: 1.2366 - val_acc: 0.6803
Epoch 200/2000
390/390 [==============================] - 7s - loss: 0.2048 - acc: 0.9487 - val_loss: 0.6845 - val_acc: 0.7673

...

390/390 [==============================] - 7s - loss: 5.0789e-04 - acc: 1.0000 - val_loss: 0.9056 - val_acc: 0.8210
Epoch 1719/2000
390/390 [==============================] - 7s - loss: 6.1475e-04 - acc: 1.0000 - val_loss: 0.9045 - val_acc: 0.8184
Epoch 1720/2000
390/390 [==============================] - 7s - loss: 0.0024 - acc: 1.0000 - val_loss: 0.8883 - val_acc: 0.8184
Epoch 1721/2000
390/390 [==============================] - 7s - loss: 0.0031 - acc: 1.0000 - val_loss: 0.9679 - val_acc: 0.8159
Epoch 1722/2000
390/390 [==============================] - 7s - loss: 0.0019 - acc: 1.0000 - val_loss: 0.8881 - val_acc: 0.8261
Epoch 1723/2000
390/390 [==============================] - 7s - loss: 0.0021 - acc: 1.0000 - val_loss: 0.9101 - val_acc: 0.8107
Epoch 1724/2000
390/390 [==============================] - 7s - loss: 0.0021 - acc: 1.0000 - val_loss: 0.8975 - val_acc: 0.8210
Epoch 1725/2000
390/390 [==============================] - 7s - loss: 7.4803e-04 - acc: 1.0000 - val_loss: 0.907

390/390 [==============================] - 7s - loss: 0.0010 - acc: 1.0000 - val_loss: 0.9543 - val_acc: 0.8107
Epoch 1781/2000
390/390 [==============================] - 7s - loss: 0.0017 - acc: 1.0000 - val_loss: 0.9269 - val_acc: 0.8133
Epoch 1782/2000
390/390 [==============================] - 7s - loss: 0.0011 - acc: 1.0000 - val_loss: 0.9782 - val_acc: 0.8159
Epoch 1783/2000
390/390 [==============================] - 7s - loss: 0.0012 - acc: 1.0000 - val_loss: 0.9268 - val_acc: 0.8159
Epoch 1784/2000
390/390 [==============================] - 7s - loss: 9.1592e-04 - acc: 1.0000 - val_loss: 0.8942 - val_acc: 0.8159
Epoch 1785/2000
390/390 [==============================] - 7s - loss: 8.5413e-04 - acc: 1.0000 - val_loss: 0.8884 - val_acc: 0.8210
Epoch 1786/2000
390/390 [==============================] - 7s - loss: 6.8575e-04 - acc: 1.0000 - val_loss: 0.8904 - val_acc: 0.8159
Epoch 1787/2000
390/390 [==============================] - 7s - loss: 8.8331e-04 - acc: 1.0000 - val_loss: 0

390/390 [==============================] - 7s - loss: 0.0023 - acc: 1.0000 - val_loss: 1.0035 - val_acc: 0.8235
Epoch 1843/2000
390/390 [==============================] - 7s - loss: 0.0089 - acc: 0.9974 - val_loss: 0.9777 - val_acc: 0.8261
Epoch 1844/2000
390/390 [==============================] - 7s - loss: 0.0102 - acc: 0.9974 - val_loss: 0.9949 - val_acc: 0.8107
Epoch 1845/2000
390/390 [==============================] - 7s - loss: 0.0065 - acc: 0.9974 - val_loss: 0.9116 - val_acc: 0.8261
Epoch 1846/2000
390/390 [==============================] - 7s - loss: 0.0060 - acc: 1.0000 - val_loss: 1.0204 - val_acc: 0.8005
Epoch 1847/2000
390/390 [==============================] - 7s - loss: 0.0035 - acc: 1.0000 - val_loss: 0.8340 - val_acc: 0.8261
Epoch 1848/2000
390/390 [==============================] - 7s - loss: 0.0012 - acc: 1.0000 - val_loss: 0.8520 - val_acc: 0.8338
Epoch 1849/2000
390/390 [==============================] - 7s - loss: 8.1060e-04 - acc: 1.0000 - val_loss: 0.8626 - val_

390/390 [==============================] - 7s - loss: 5.2379e-04 - acc: 1.0000 - val_loss: 0.9332 - val_acc: 0.8184
Epoch 1905/2000
390/390 [==============================] - 7s - loss: 5.3403e-04 - acc: 1.0000 - val_loss: 0.9303 - val_acc: 0.8210
Epoch 1906/2000
390/390 [==============================] - 7s - loss: 5.1710e-04 - acc: 1.0000 - val_loss: 0.9300 - val_acc: 0.8210
Epoch 1907/2000
390/390 [==============================] - 7s - loss: 0.0014 - acc: 1.0000 - val_loss: 0.9171 - val_acc: 0.8235
Epoch 1908/2000
390/390 [==============================] - 7s - loss: 0.0059 - acc: 0.9974 - val_loss: 0.9687 - val_acc: 0.8159
Epoch 1909/2000
390/390 [==============================] - 7s - loss: 0.0032 - acc: 1.0000 - val_loss: 1.0877 - val_acc: 0.7724
Epoch 1910/2000
390/390 [==============================] - 7s - loss: 0.0022 - acc: 1.0000 - val_loss: 0.9096 - val_acc: 0.8338
Epoch 1911/2000
390/390 [==============================] - 7s - loss: 9.5580e-04 - acc: 1.0000 - val_loss: 0

390/390 [==============================] - 7s - loss: 6.5170e-04 - acc: 1.0000 - val_loss: 0.9070 - val_acc: 0.8107
Epoch 1968/2000
390/390 [==============================] - 7s - loss: 8.1560e-04 - acc: 1.0000 - val_loss: 0.9111 - val_acc: 0.8107
Epoch 1969/2000
390/390 [==============================] - 7s - loss: 0.0010 - acc: 1.0000 - val_loss: 0.9197 - val_acc: 0.8184
Epoch 1970/2000
390/390 [==============================] - 7s - loss: 8.7603e-04 - acc: 1.0000 - val_loss: 0.9259 - val_acc: 0.8210
Epoch 1971/2000
390/390 [==============================] - 7s - loss: 6.0925e-04 - acc: 1.0000 - val_loss: 0.9338 - val_acc: 0.8210
Epoch 1972/2000
390/390 [==============================] - 7s - loss: 4.8815e-04 - acc: 1.0000 - val_loss: 0.9234 - val_acc: 0.8210
Epoch 1973/2000
390/390 [==============================] - 7s - loss: 0.0012 - acc: 1.0000 - val_loss: 0.9163 - val_acc: 0.8184
Epoch 1974/2000
390/390 [==============================] - 7s - loss: 6.9285e-04 - acc: 1.0000 - val

In [13]:
#result = log.append(['min_loss',log.loc[log['loss'].idxmin]['loss'] ],['val_acc',log.loc[log['loss'].idxmin]['val_acc']])
s = pd.Series({'acc':"min_loss", 'loss':log.loc[log['loss'].idxmin]['loss'], 'lr':"val_acc",'val_acc':log.loc[log['loss'].idxmin]['val_acc']})
# 这里 Series 必须是 dict-like 类型
result = log.append(s, ignore_index=True)
# 这里必须选择ignore_index=True 或者给 Series 一个index值
result.to_csv('5_layers_result.csv')
result

,acc,loss,lr,val_acc,val_loss
0,0.074359,3.573387,0.001,0.086957,3.600555
1,0.110256,3.238511,0.001,0.028133,3.665757
2,0.202564,3.038554,0.001,0.028133,4.578237
3,0.241026,2.899888,0.001,0.028133,7.068300
4,0.269231,2.765142,0.001,0.028133,9.497924
5,0.305128,2.624422,0.001,0.028133,10.772123
6,0.346154,2.500278,0.001,0.028133,9.801348
7,0.407692,2.419845,0.001,0.028133,12.562832
8,0.412821,2.359174,0.001,0.028133,12.154218
9,0.451282,2.231364,0.001,0.028133,11.424444


In [15]:
result.to_csv('5_layers_result.csv')